### Schedule duration monitor

This notebook will monitor - in live time - the duration of the expeditions in directories GROUP1, GROUP2, ... , GROUP66.

The resultant plot once all cells of the notebook are run will be refreshed every n seconds (prescribable in the `REFRESH` constant below).

CTDs are assumed to take approx. 20 minutes each. 3 days sailing time is added to all groups for the outbound journey from Texel to the first waypoint.

<div class="alert alert-warning">
This script uses an infinite `while` loop to refresh the plotting indefinitely. To stop the running, the "Interupt the kernel" button (square button) should be pressed.
</div>


In [1]:
import time
import os
import sys
import yaml
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [2]:
# plot refresh rate
REFRESH = 30  # [seconds]

In [3]:
# config
BASE_DIR = Path("/home/shared/data/virtualship_storage/")
SAIL_OUT_TIME = np.timedelta64(3, "D")  # [days]
CTD_TIME = np.timedelta64(200, "m")  # [minutes]
SHIP_TIME_THRESHOLD = 9  # [days]
ROTATION_CHANGE = 18  # when to change from 45 to 90 degree rotation in x axis labels

In [4]:
def preprocess(
    base_dir: Path, sail_out_time: np.timedelta64, ctd_time: np.timedelta64
) -> dict:
    """
    Reads schedule data from YAML files, calculates the total expedition duration
    for each group, and returns a dictionary of valid groups and their durations.
    """
    groups = {}

    # group directories 1 to 66 (inclusive)
    for i in range(1, 67):
        group_name = f"GROUP{i}"
        group_dir = base_dir / group_name
        schedule_file = group_dir / "schedule.yaml"

        if not schedule_file.exists():
            groups[group_name] = np.nan
            continue

        try:
            with open(schedule_file, "r", encoding="utf-8") as f:
                schedule_data = yaml.safe_load(f)

            waypoints = schedule_data.get("waypoints", [])

            if not waypoints:
                groups[group_name] = np.nan
                continue

            start_time_str = waypoints[0].get("time")
            end_time_str = waypoints[-1].get("time")

            if not start_time_str or not end_time_str:
                groups[group_name] = np.nan
            else:
                start_time, end_time = (
                    np.datetime64(start_time_str),
                    np.datetime64(end_time_str),
                )

                difference = end_time - start_time
                difference_days = difference.astype("timedelta64[D]")  # [days]

                total_time_timedelta = difference_days + sail_out_time + ctd_time
                total_time_days = total_time_timedelta.astype(
                    "timedelta64[h]"
                ) / np.timedelta64(24, "h")

                groups[group_name] = total_time_days.item()

        except Exception as e:
            groups[group_name] = np.nan
            print(f"Error processing {group_name}: {e}", file=sys.stderr)

    # filter dict to remove groups with NaN
    filter_groups = {key: value for key, value in groups.items() if not np.isnan(value)}

    return filter_groups

In [ ]:
def plot(filter_groups: dict, ship_time_threshold: float, rotation_change: int):
    """
    Generates a bar plot showing the expedition duration for each group
    relative to the ship time threshold.
    """
    groups_keys = list(filter_groups.keys())
    groups_values = list(filter_groups.values())

    # bar colors dependent on whether above or below ship time threshold
    bar_colors = [
        "crimson" if value > ship_time_threshold else "mediumseagreen"
        for value in groups_values
    ]

    # fig
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 8), dpi=300)

    # bars
    ax.bar(
        groups_keys,
        groups_values,
        color=bar_colors,
        edgecolor="k",
        linewidth=1.5,
        zorder=3,
        width=0.7,
    )

    # labels and title
    ax.set_ylabel("Days", fontsize=15)
    ax.set_title("Expedition Duration", fontsize=20)

    # customise ticks
    ax.set_xticks(ax.get_xticks())
    rotation = 45 if len(groups_values) <= rotation_change else 90
    ax.set_xticklabels(groups_keys, rotation=rotation, ha="center", fontsize=15)
    ax.tick_params(axis="y", labelsize=15)

    # set y-limit based on the maximum valid value
    max_duration = np.nanmax(groups_values) if groups_values else 0
    ax.set_ylim(0, max_duration + 0.5)

    # grid
    ax.set_facecolor("gainsboro")
    ax.grid(axis="y", linestyle="-", alpha=1.0, color="white")

    # horizontal line for threshold days
    ax.axhline(
        y=ship_time_threshold,
        color="r",
        linestyle="--",
        linewidth=2.5,
        label="Ship-time limit",
        zorder=2,
    )

    plt.legend(fontsize=15, loc="upper left")
    plt.tight_layout()
    plt.show()

In [6]:
def periodic_task(interval_seconds: int):
    """
    Loop that runs the preprocessing and plotting functions periodically.
    :param interval_seconds: The number of seconds to wait between runs.
    """
    while True:
        try:
            clear_output(wait=True)
            print(f"--- Running task at {time.ctime()} ---")

            data = preprocess(BASE_DIR, SAIL_OUT_TIME, CTD_TIME)

            if data:
                plot(data, SHIP_TIME_THRESHOLD, ROTATION_CHANGE)
            else:
                print("No valid data found to plot.")

        except KeyboardInterrupt:
            print("\nPeriodic task interrupted by user.")
            break
        except Exception as e:
            print(f"\nAn error occurred during the periodic run: {e}")

        time.sleep(interval_seconds)

In [7]:
periodic_task(interval_seconds=REFRESH)

--- Running task at Wed Nov 12 21:45:02 2025 ---
No valid data found to plot.


KeyboardInterrupt: 